In [ ]:
print("Kernel check")

# LifeTrac v25 Design Compliance Review

This notebook parses the OpenSCAD parameter files and validates the design against the rules defined in `DESIGN_RULES.md`.

## 1. Environment Setup & File Loading


In [ ]:
import re
import os
import math

# Define paths
base_path = r"c:\Users\dorkm\Documents\GitHub\LifeTrac\LifeTrac-v25\mechanical_design\openscad"
params_file = os.path.join(base_path, "lifetrac_v25_params.scad")
assembly_file = os.path.join(base_path, "lifetrac_v25.scad")
arm_plate_file = os.path.join(base_path, "parts", "arm_plate.scad")

# Helper to read file
def read_scad(path):
    with open(path, 'r') as f:
        return f.read()

# Helper to extract variable value (simple scalar or calculation)
# Note: This is a simple regex parser, not a full SCAD interpreter. 
# It works for lines like "VAR = 123;" or "VAR = 12 * 25.4;"
def get_var(content, var_name):
    # Match: VAR = value; (ignoring comments)
    pattern = r"^\s*" + re.escape(var_name) + r"\s*=\s*([^;]+);"
    match = re.search(pattern, content, re.MULTILINE)
    if match:
        val_str = match.group(1).split("//")[0].strip() # Remove EOL comments
        # Attempt to eval simple math
        try:
            # Replace common SCAD constants if needed, or just eval python math
            val_str = val_str.replace("PLATE_1_4_INCH", "6.35")
            val_str = val_str.replace("PLATE_1_2_INCH", "12.7")
            val_str = val_str.replace("PLATE_3_4_INCH", "19.05")
            return eval(val_str, {"__builtins__": None, "max": max, "min": min, "pow": pow, "sqrt": math.sqrt})
        except:
            return val_str # Return string if complex (e.g., function call)
    return None

# Load contents
params_content = read_scad(params_file)
arm_plate_content = read_scad(arm_plate_file)

print("Files loaded successfully.")


## 2. Rule Validation Logic

We now check each rule from the `DESIGN_RULES.md` file.

### Rule 1: Bucket Geometry & Ground Engagement
*   **Zero-Z Ground Contact**: `BUCKET_GROUND_CLEARANCE` <= 0.
*   **1/5th Pivot Height Rule**: `BUCKET_PIVOT_HEIGHT_FROM_BOTTOM` should essentially equate to `BUCKET_HEIGHT / 5`. Since SCAD variables might be formulas, we check the extracted values.


In [ ]:
results = {}

# --- Rule 1 Checks ---
r1_ground_clearance = get_var(params_content, "BUCKET_GROUND_CLEARANCE")
r1_bucket_height = get_var(params_content, "BUCKET_HEIGHT")
r1_pivot_height = get_var(params_content, "BUCKET_PIVOT_HEIGHT_FROM_BOTTOM")

# Check 1: Zero-Z Ground Contact
if r1_ground_clearance is not None and r1_ground_clearance <= 0:
    results["R1.1_Ground_Contact"] = "PASS"
else:
    results["R1.1_Ground_Contact"] = f"FAIL (Value: {r1_ground_clearance})"

# Check 2: 1/5th Height Rule
if r1_bucket_height and r1_pivot_height:
    expected = r1_bucket_height / 5
    # Allow small float diff
    if abs(r1_pivot_height - expected) < 0.01:
        results["R1.2_Pivot_Height_Ratio"] = "PASS"
    else:
        results["R1.2_Pivot_Height_Ratio"] = f"FAIL (Expected {expected}, Got {r1_pivot_height})"
else:
    results["R1.2_Pivot_Height_Ratio"] = "ERROR (Could not parse variables)"

results


### Rule 2 & 3: Kinematics & Alignment
*   **Target Priority**: Check if `_T_z` uses the pivot height directly (no arbitrary clamps like `max(50, ...)`).
*   **Matching Tip Radii**: Check if `boss_r` in `arm_plate.scad` uses the `PIVOT_HOLE_X_FROM_FRONT` parameter.
*   **Hole Alignment**: Check `cx` and `cz` in `arm_plate.scad`.


In [ ]:
# --- Rule 2: Target Geometry ---
# We have to inspect the text logic for `_T_z` in params_content
# Look for: _T_z = BUCKET_PIVOT_HEIGHT_FROM_BOTTOM + BUCKET_GROUND_CLEARANCE;
# Avoid: max(50, ...
if "max(50" in params_content and "_T_z" in params_content:
     # We need to verify if max(50 is applied to _T_z line specifically
     tz_line_match = re.search(r"_T_z\s*=\s*(.*);", params_content)
     if tz_line_match and "max(50" in tz_line_match.group(1):
         results["R2.1_Target_Priority"] = "FAIL (Arbitrary clamp found)"
     else:
         results["R2.1_Target_Priority"] = "PASS"
else:
    results["R2.1_Target_Priority"] = "PASS"

# --- Rule 3: Component Alignment ---
# Check arm_plate.scad for boss_r definition
# It should be: boss_r = PIVOT_HOLE_X_FROM_FRONT;
boss_r_match = re.search(r"boss_r\s*=\s*PIVOT_HOLE_X_FROM_FRONT", arm_plate_content)
if boss_r_match:
    results["R3.1_Matching_Tip_Radii"] = "PASS"
else:
    results["R3.1_Matching_Tip_Radii"] = "FAIL (boss_r not linked to PIVOT_HOLE_X_FROM_FRONT)"

# Check hole alignment logic in arm_plate.scad
# cx = drop_ext - PIVOT_HOLE_X_FROM_FRONT
cx_match = re.search(r"cx\s*=\s*drop_ext\s*-\s*PIVOT_HOLE_X_FROM_FRONT", arm_plate_content)
if cx_match:
    results["R3.2_Hole_X_Alignment"] = "PASS"
else:
    results["R3.2_Hole_X_Alignment"] = "FAIL (cx logic mismatch)"

results


### Rule 6: Standard Materials & Safety
*   **Standard Materials**: Check if `TUBE_` variables are used.
*   **Hole Edge Margin**: `PIVOT_HOLE_X_FROM_FRONT` (boss radius/center pos) - `BUCKET_PIVOT_PIN_DIA/2` > `6.35`?


In [ ]:
# --- Rule 6 Checks ---

# Material Safety Margin Check
# Margin = Radius of Steel - Radius of Hole
# Radius of Steel (Tip) = PIVOT_HOLE_X_FROM_FRONT
# Radius of Hole = BUCKET_PIVOT_PIN_DIA / 2

steel_radius = get_var(params_content, "PIVOT_HOLE_X_FROM_FRONT")
pin_dia = get_var(params_content, "BUCKET_PIVOT_PIN_DIA") # 25.4

if steel_radius and pin_dia:
    margin = steel_radius - (pin_dia / 2)
    min_margin = 6.35 # 0.25 inch
    
    if margin >= min_margin:
        results["R6.2_Hole_Edge_Margin"] = f"PASS (Margin: {margin:.2f}mm >= {min_margin}mm)"
    else:
        results["R6.2_Hole_Edge_Margin"] = f"FAIL (Margin: {margin:.2f}mm < {min_margin}mm)"
else:
    results["R6.2_Hole_Edge_Margin"] = "ERROR (Could not calc margin)"

# Standard Tube Check
# Just check if we define TUBE_2X6_1_4
if "TUBE_2X6_1_4" in params_content and "TUBE_3X3_1_4" in params_content:
     results["R6.3_Standard_Materials"] = "PASS"
else:
     results["R6.3_Standard_Materials"] = "FAIL (Missing standard definitions)"

results
